## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import dataframe_image as dfi # To conveinently save DFs to image file

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Krasnoselkup,RU,2021-12-19 15:22:47,65.7000,82.4667,-41.21,98,27,4.25,scattered clouds
1,1,Ipixuna,BR,2021-12-19 15:22:47,-1.7625,-48.8043,87.75,86,100,3.94,light rain
2,2,Cape Town,ZA,2021-12-19 15:22:48,-33.9258,18.4232,82.11,60,0,5.01,clear sky
3,3,Norman Wells,CA,2021-12-19 15:22:48,65.2820,-126.8329,-0.36,85,90,2.30,light snow
4,4,Verkh-Chebula,RU,2021-12-19 15:22:49,56.0308,87.6218,-11.09,100,96,3.51,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
lowest_temp = float(input("What is lowest temperature you would accept for your vacation? "))
highest_temp = float(input("What is lowest temperature you would accept for your vacation? "))


What is lowest temperature you would accept for your vacation? 79
What is lowest temperature you would accept for your vacation? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= highest_temp) &
                                       (city_data_df["Max Temp"] >= lowest_temp)]
print(f'There were {len(matching_cities_df)} cites that met your temperture profile.')
matching_cities_df

There were 18 cites that met your temperture profile.


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
62,62,Mongagua,BR,2021-12-19 15:20:44,-24.0931,-46.6208,79.93,79,93,7.40,overcast clouds
94,94,Faanui,PF,2021-12-19 15:25:37,-16.4833,-151.7500,79.59,76,70,8.39,broken clouds
170,170,Port Elizabeth,ZA,2021-12-19 15:27:14,-33.9180,25.5701,79.11,73,0,21.85,clear sky
181,181,Chuy,UY,2021-12-19 15:27:21,-33.6971,-53.4616,79.03,48,0,13.58,clear sky
306,306,Hambantota,LK,2021-12-19 15:31:22,6.1241,81.1185,79.14,94,40,6.91,scattered clouds
319,319,Namatanai,PG,2021-12-19 15:31:29,-3.6667,152.4333,79.63,82,99,1.28,overcast clouds
368,368,Humaita,BR,2021-12-19 15:33:52,-7.5061,-63.0208,79.25,91,100,3.76,overcast clouds
410,410,Miracema Do Tocantins,BR,2021-12-19 15:34:16,-9.5618,-48.3967,79.23,88,100,4.32,overcast clouds
447,447,Fare,PF,2021-12-19 15:35:33,-16.7000,-151.0167,79.23,78,69,8.30,broken clouds
504,504,Buala,SB,2021-12-19 15:37:13,-8.1450,159.5921,79.11,85,99,6.15,light rain


In [5]:
# 4a. Determine if there are any empty rows.
matching_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
matching_cities_df = matching_cities_df.dropna()

print(f"There were {len(matching_cities_df)} cities that matched your temperature profile.")
matching_cities_df.head()

There were 18 cities that matched your temperature profile.


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
62,62,Mongagua,BR,2021-12-19 15:20:44,-24.0931,-46.6208,79.93,79,93,7.40,overcast clouds
94,94,Faanui,PF,2021-12-19 15:25:37,-16.4833,-151.7500,79.59,76,70,8.39,broken clouds
170,170,Port Elizabeth,ZA,2021-12-19 15:27:14,-33.9180,25.5701,79.11,73,0,21.85,clear sky
181,181,Chuy,UY,2021-12-19 15:27:21,-33.6971,-53.4616,79.03,48,0,13.58,clear sky
306,306,Hambantota,LK,2021-12-19 15:31:22,6.1241,81.1185,79.14,94,40,6.91,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = matching_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
62,Mongagua,BR,79.93,overcast clouds,-24.0931,-46.6208,
94,Faanui,PF,79.59,broken clouds,-16.4833,-151.7500,
170,Port Elizabeth,ZA,79.11,clear sky,-33.9180,25.5701,
181,Chuy,UY,79.03,clear sky,-33.6971,-53.4616,
306,Hambantota,LK,79.14,scattered clouds,6.1241,81.1185,
319,Namatanai,PG,79.63,overcast clouds,-3.6667,152.4333,
368,Humaita,BR,79.25,overcast clouds,-7.5061,-63.0208,
410,Miracema Do Tocantins,BR,79.23,overcast clouds,-9.5618,-48.3967,
447,Fare,PF,79.23,broken clouds,-16.7000,-151.0167,
504,Buala,SB,79.11,light rain,-8.1450,159.5921,


In [8]:
# # Set the parameters to search for a hotel in Paris.
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "key": google_api_key,
#     "location": "48.8566, 2.3522"}
# # Use base URL to search for hotels in Paris.
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# # Make request and get the JSON data from the search.
# hotels = requests.get(base_url, params=params).json()

# hotels

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
count_hotels = 0
count_no_hotels = 0

params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        count_hotels += 1

    except (IndexError):
        print("Hotel not found...Skipping City.")
        count_no_hotels += 1
        
# How many hotels were found
print(f"From a total of {count_hotels + count_no_hotels} cities searched there were a total of {count_hotels} hotels found.")
        

From a total of 18 cities searched there were a total of 18 hotels found.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
dfi.export(hotel_df.head(10), 'Table1.png')
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
62,Mongagua,BR,79.93,overcast clouds,-24.0931,-46.6208,Hostel Aloha Brasil
94,Faanui,PF,79.59,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
170,Port Elizabeth,ZA,79.11,clear sky,-33.9180,25.5701,39 On Nile Guest House
181,Chuy,UY,79.03,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
306,Hambantota,LK,79.14,scattered clouds,6.1241,81.1185,Bungalow 63
319,Namatanai,PG,79.63,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
368,Humaita,BR,79.25,overcast clouds,-7.5061,-63.0208,Corrêa Hotel
410,Miracema Do Tocantins,BR,79.23,overcast clouds,-9.5618,-48.3967,Miami Apart Hotel
447,Fare,PF,79.23,broken clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
504,Buala,SB,79.11,light rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))